In [1]:
import torch
from ModelAi.projectFeature import PJFTrainModel, PJF
import pandas as pd

PJF_model = PJF()
PJF_model.to("cuda")
model = PJFTrainModel(model=PJF_model)
model.to("cuda")

model.load_state_dict(
    torch.load(
        f"D:\\AIChallenge\\Checkpoints\\model_epoch_{2}.pt",
    )
)

PJF_model = model.model
PJF_model.eval()

PJF(
  (dropout): Dropout(p=0.4, inplace=False)
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attention_2): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (atte_reduce): AttentionReduce(
    (mlp): Sequential(
      (0): Linear(in_features=768, out_features=384, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=384, out_features=1, bias=True)
    )
  )
  (ffn_2): Sequential(
    (0): Linear(in_features=768, out_features=1536, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1536, out_features=4096, bias=True)
  )
)

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

# BARTpho-syllable
# syllable_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
# bartpho_syllable = AutoModel.from_pretrained("vinai/bartpho-syllable")
# TXT = "Chúng tôi là những nghiên cứu viên."
# input_ids = syllable_tokenizer(TXT, return_tensors="pt")["input_ids"]
# features = bartpho_syllable(input_ids)

# BARTpho-word
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word-base")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word-base")
# to cuda
bartpho_word.to("cuda")
bartpho_word.eval()
for param in bartpho_word.parameters():
    param.requires_grad = False
# TXT = "Chúng_tôi là những nghiên_cứu_viên ."
# input_ids = word_tokenizer(TXT, return_tensors="pt")["input_ids"]
# features = bartpho_word(input_ids.to("cuda"))

# input_ids = word_tokenizer.encode(TXT, return_tensors="pt").to("cuda")
# print(input_ids)
# # check if input_ids is in cuda
# features = bartpho_word(input_ids).last_hidden_state[0]

d:\AIChallenge\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\AIChallenge\venv\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
# Word segmentation
import py_vncorenlp

# py_vncorenlp.download_model(save_dir=r"D:\VQA\Notebook\VnCoreNLP")

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(
    annotators=["wseg"],
    save_dir=r"D:\AIChallenge\VnCoreNLP",
)

# change dir to C:\Users\Asus.LAPTOP-8EU9PHJL.000\Desktop\VQA\Notebook
import os

os.chdir(r"D:\AIChallenge")


def Word_Segmentation(text):
    output = rdrsegmenter.word_segment(text)
    if len(output) > 0:
        # join all words in the list
        res = ""
        for i in output:
            res += i + " "
        return res
    else:
        output = ""
    return output


print(Word_Segmentation("Chúng tôi là những nghiên cứu viên."))

Chúng_tôi là những nghiên_cứu_viên . 


In [4]:
import os
from tqdm import tqdm
import torch

def scan_file_path_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            lst.append(os.path.join(root, file))
    return lst


def scan_file_name_in_folder(directory):
    lst = []
    for root, dirs, files in os.walk(directory):
        # check files in the directory
        for file in files:
            # remove the extension
            file_name = os.path.splitext(file)[0]
            lst.append(file_name)
    return lst

In [5]:
# audio
audio_path = "Data\\PreporcessData\\Projection\\Audio_text_bartpho"
audio_path_lst = scan_file_path_in_folder(audio_path)
audio_name_lst = scan_file_name_in_folder(audio_path)
audio_path_dict = dict(zip(audio_name_lst, audio_path_lst))
# audio key
audio_key_path = "Data\\PreporcessData\\Projection\\Audio_key_text_bartpho"
audio_key_path_lst = scan_file_path_in_folder(audio_key_path)
audio_key_name_lst = scan_file_name_in_folder(audio_key_path)
audio_key_path_dict = dict(zip(audio_key_name_lst, audio_key_path_lst))
# key image caption
key_image_caption_path = (
    "Data\\PreporcessData\\Projection\\keyframes_ImageCaption_bartpho"
)
key_image_caption_path_lst = scan_file_path_in_folder(key_image_caption_path)
key_image_caption_name_lst = scan_file_name_in_folder(key_image_caption_path)
key_image_caption_path_dict = dict(
    zip(key_image_caption_name_lst, key_image_caption_path_lst)
)

In [6]:
print(len(audio_path_dict), len(audio_key_path_dict), len(key_image_caption_path_dict))

# map
video_frame_lst = {}
for key in audio_name_lst:
    for key_key in audio_key_name_lst:
        if key in key_key:
            if key in video_frame_lst:
                video_frame_lst[key].append(key_key)
            else:
                video_frame_lst[key] = [key_key]
print(len(video_frame_lst))
print(video_frame_lst["L01_V001"])

363 106589 106589
363
['L01_V001_001', 'L01_V001_002', 'L01_V001_003', 'L01_V001_004', 'L01_V001_005', 'L01_V001_006', 'L01_V001_007', 'L01_V001_008', 'L01_V001_009', 'L01_V001_010', 'L01_V001_011', 'L01_V001_012', 'L01_V001_013', 'L01_V001_014', 'L01_V001_015', 'L01_V001_016', 'L01_V001_017', 'L01_V001_018', 'L01_V001_019', 'L01_V001_020', 'L01_V001_021', 'L01_V001_022', 'L01_V001_023', 'L01_V001_024', 'L01_V001_025', 'L01_V001_026', 'L01_V001_027', 'L01_V001_028', 'L01_V001_029', 'L01_V001_030', 'L01_V001_031', 'L01_V001_032', 'L01_V001_033', 'L01_V001_034', 'L01_V001_035', 'L01_V001_036', 'L01_V001_037', 'L01_V001_038', 'L01_V001_039', 'L01_V001_040', 'L01_V001_041', 'L01_V001_042', 'L01_V001_043', 'L01_V001_044', 'L01_V001_045', 'L01_V001_046', 'L01_V001_047', 'L01_V001_048', 'L01_V001_049', 'L01_V001_050', 'L01_V001_051', 'L01_V001_052', 'L01_V001_053', 'L01_V001_054', 'L01_V001_055', 'L01_V001_056', 'L01_V001_057', 'L01_V001_058', 'L01_V001_059', 'L01_V001_060', 'L01_V001_061', '

In [7]:
# print first 5 key, value in the dictionary
print({k: audio_path_dict[k] for k in list(audio_path_dict)[:5]})
print({k: audio_key_path_dict[k] for k in list(audio_key_path_dict)[:5]})
print(
    {k: key_image_caption_path_dict[k] for k in list(key_image_caption_path_dict)[:5]}
)

{'L01_V001': 'Data\\PreporcessData\\Projection\\Audio_text_bartpho\\L01_V001.pt', 'L01_V002': 'Data\\PreporcessData\\Projection\\Audio_text_bartpho\\L01_V002.pt', 'L01_V003': 'Data\\PreporcessData\\Projection\\Audio_text_bartpho\\L01_V003.pt', 'L01_V004': 'Data\\PreporcessData\\Projection\\Audio_text_bartpho\\L01_V004.pt', 'L01_V005': 'Data\\PreporcessData\\Projection\\Audio_text_bartpho\\L01_V005.pt'}
{'L01_V001_001': 'Data\\PreporcessData\\Projection\\Audio_key_text_bartpho\\L01_V001_001.pt', 'L01_V001_002': 'Data\\PreporcessData\\Projection\\Audio_key_text_bartpho\\L01_V001_002.pt', 'L01_V001_003': 'Data\\PreporcessData\\Projection\\Audio_key_text_bartpho\\L01_V001_003.pt', 'L01_V001_004': 'Data\\PreporcessData\\Projection\\Audio_key_text_bartpho\\L01_V001_004.pt', 'L01_V001_005': 'Data\\PreporcessData\\Projection\\Audio_key_text_bartpho\\L01_V001_005.pt'}
{'L01_V001_001': 'Data\\PreporcessData\\Projection\\keyframes_ImageCaption_bartpho\\L01_V001_001.pt', 'L01_V001_002': 'Data\\Pre

In [8]:
import torch
import concurrent.futures
from tqdm import tqdm


def process_paths(data_dict):
    processed_dict = {}
    for key, path in data_dict.items():
        # to GPU
        tensor = torch.load(path, map_location="cuda")
        processed_dict[key] = {
            # "path": path,
            "tensor": tensor,
            # "square_sum_tensor": torch.sum(tensor**2),
        }
    return processed_dict


def load_data_parallel(data_dict):
    processed_dict = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(process_paths, {key: path}): key
            for key, path in data_dict.items()
        }
        for future in tqdm(
            concurrent.futures.as_completed(futures), total=len(data_dict)
        ):
            result = future.result()
            processed_dict.update(result)
    return processed_dict


# Load all data in parallel into RAM
video_dict = load_data_parallel(audio_path_dict)
audio_key_dict = load_data_parallel(audio_key_path_dict)
key_frame_dict = load_data_parallel(key_image_caption_path_dict)

100%|██████████| 106589/106589 [00:50<00:00, 2090.19it/s]


In [9]:
query_path = "pack1-groupA"
query_path_lst = scan_file_path_in_folder(query_path)
query_path_name_lst = scan_file_name_in_folder(query_path)
print(query_path_lst)

['pack1-groupA\\query-p1-1-qa.txt', 'pack1-groupA\\query-p1-10-kis.txt', 'pack1-groupA\\query-p1-11-kis.txt', 'pack1-groupA\\query-p1-12-kis.txt', 'pack1-groupA\\query-p1-13-kis.txt', 'pack1-groupA\\query-p1-14-kis.txt', 'pack1-groupA\\query-p1-15-kis.txt', 'pack1-groupA\\query-p1-16-kis.txt', 'pack1-groupA\\query-p1-17-kis.txt', 'pack1-groupA\\query-p1-18-kis.txt', 'pack1-groupA\\query-p1-19-kis.txt', 'pack1-groupA\\query-p1-2-qa.txt', 'pack1-groupA\\query-p1-20-kis.txt', 'pack1-groupA\\query-p1-21-kis.txt', 'pack1-groupA\\query-p1-22-kis.txt', 'pack1-groupA\\query-p1-23-kis.txt', 'pack1-groupA\\query-p1-24-qa.txt', 'pack1-groupA\\query-p1-25-qa.txt', 'pack1-groupA\\query-p1-3-qa.txt', 'pack1-groupA\\query-p1-4-kis.txt', 'pack1-groupA\\query-p1-5-kis.txt', 'pack1-groupA\\query-p1-6-kis.txt', 'pack1-groupA\\query-p1-7-kis.txt', 'pack1-groupA\\query-p1-8-kis.txt', 'pack1-groupA\\query-p1-9-kis.txt']


In [10]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")

Failed to create new connection using: 5f9cc94976404116a2c3a87dc717f624


ModuleNotFoundError: No module named 'milvus_lite'